# Setup section

In [ ]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
import openai

# Prompt section

Examples used for the one-shot and few-shot prompting

In [ ]:
# ex12
ex1_input = """
The company is organised into departments. Each department has a unique name, a unique number, and a particular employee who manages the department.
We keep track of the start date when that employee began managing the department. A department may have several locations.
A department controls a number of projects, each of which has a unique name, a unique number, and a single location.
We store each employee’s name, Social Security number, address, salary, sex (gender), and birth date.
An employee is assigned to one department, but may work on several projects, which are not necessarily controlled by the same department.
We keep track of the current number of hours per week that an employee works on each project. We also keep track of the direct supervisor of each employee (who is another employee).
We want to keep track of the dependents of each employee for insurance purposes. We keep each dependent’s first name, sex, number, birth date, and relationship to the employee.
"""

ex1_output = """
Each department has a name and number. The department’s name is unique. The department’s number is unique.
The department is managed by a unique employee. The managing has a starting date. A department may have several locations.
A department controls zero or more projects. The project has a name, number, and location. The project’s name is unique.
The project’s number is unique. The employee has a Social Security Number, address, salary, sex, and birth date.
An employee is assigned to one department. An employee can work on several projects. The working has a current number of hours per week.
An employee can supervise zero or many employees. An employee can have one or more dependents. A dependent has a first name, sex, number, birth date, and relationship.
"""

# ex11
ex2_input = """
The database must store painters, paintings, and galleries information. Painters have a unique number, Name, and phone number.
Paintings have a unique number, title and price. Galleries have a unique number, owner, phone number, commission rate and address.
A painting is painted by a particular artist, and that painting is exhibited in a particular gallery.
A gallery can exhibit many paintings, but each painting can be exhibited in only one gallery.
Similarity, a painting is painted by a single painter, but each painter can paint many paintings.
"""

ex2_output = """
The database must store painters, paintings, and galleries.
Painters have a number, name, and phone number. The painter’s number is unique. Paintings have a number, title and price.
The painting’s number is unique. Galleries have a number, owner, phone number, commission rate and address.
The gallerie’s number is unique. A painting is painted by a unique artist, and the painting is exhibited in a unique gallery.
A gallery can exhibit many paintings, but each painting can be exhibited in only one gallery.
A painting is painted by a single painter, but each painter can paint many paintings.
"""

One-shot prompting

Problems:
- put 'unique' attributes in a separated sentence (easier to handle)

In [ ]:
# Key that gives an access to openai account
openai.api_key = 'key_code'

# parameter controlling randomness of the output
temp = 0.0


def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


text = """The database must store painters, paintings, and galleries information.
Painters have a unique number, Name, and phone number. Paintings have a unique number, title and price.
Galleries have a unique number, owner, phone number, commission rate and address.
A painting is painted by a particular artist, and that painting is exhibited in a particular gallery.
A gallery can exhibit many paintings, but each painting can be exhibited in only one gallery.
Similarity, a painting is painted by a single painter, but each painter can paint many paintings.
"""

correct_text = """
The database must store painters, paintings, and galleries.
Painters have a number, name, and phone number. The painter’s number is unique.
Paintings have a number, title and price. The painting’s number is unique.
Galleries have a number, owner, phone number, commission rate and address. The gallerie’s number is unique.
A painting is painted by a unique artist, and the painting is exhibited in a unique gallery.
A gallery can exhibit many paintings, but each painting can be exhibited in only one gallery.
Each painter can paint many paintings.
"""

prompt = f"""
The task is to extract an Entity-relationship diagram from natural language requirements.
To do that, these requirements should be simplified as much as possible for the extraction of entities,
attributes and relationships (i.e. short and simple sentences, explicit subjects, etc.).
two examples of semplification are given below, where the 'input text' is the original text and the 'output text' is the simplified text:

Input text:
'
{ex1_input}
'

Output text:
'
{ex1_output}
'

Simplify as shown above the following text delimited by triple backticks, remembering to not remove any information and creating short and simple sentences:

Text:
```{text}```
"""

answers = []
print("___________")
for i in range(1):
    response = get_completion(prompt)
    answers.append(response)

for x in answers:
    print(x)

    print("___________")

print('Correct output text:\n', correct_text)

___________
Output text:
```
The database stores painters, paintings, and galleries. 
A painter has a unique number, name, and phone number. 
A painting has a unique number, title, and price. 
A gallery has a unique number, owner, phone number, commission rate, and address. 
A painting is painted by one painter. 
A painting is exhibited in one gallery. 
A gallery can exhibit many paintings. 
A painter can paint many paintings.
```
___________
Correct output text:
 
The database must store painters, paintings, and galleries.
Painters have a number, name, and phone number. The painter’s number is unique.
Paintings have a number, title and price. The painting’s number is unique.
Galleries have a number, owner, phone number, commission rate and address. The gallerie’s number is unique.
A painting is painted by a unique artist, and the painting is exhibited in a unique gallery.
A gallery can exhibit many paintings, but each painting can be exhibited in only one gallery.
Each painter can pai

Few-shot prompting - attempt 1

Problems:
- cannot recognize and avoid repetitive sentences with synonyms (e.g. A painting is painted by one artist, A painting is painted by one painter)

In [ ]:
text = """The database must store painters, paintings, and galleries information.
Painters have a unique number, Name, and phone number. Paintings have a unique number, title and price.
Galleries have a unique number, owner, phone number, commission rate and address.
A painting is painted by a particular artist, and that painting is exhibited in a particular gallery.
A gallery can exhibit many paintings, but each painting can be exhibited in only one gallery.
Similarity, a painting is painted by a single painter, but each painter can paint many paintings.
"""

prompt = f"""
The task is to extract an Entity-relationship diagram from natural language requirements.
To do that, these requirements should be simplified as much as possible for the extraction of entities,
attributes and relationships (i.e. short and simple sentences, explicit subjects, etc.).
Two examples of semplification are given below, where the 'input text' is the original text and the 'output text' is the simplified text:

Input text:
'
{ex1_input}
'

Output text:
'
{ex1_output}
'

Input text:
'
{ex2_input}
'

Output text:
'
{ex2_output}
'

Simplify as shown above the following text delimited by triple backticks, remembering to not remove any information and creating short and simple sentences:

Text:
```{text}```
"""

answers = []
print("___________")
for i in range(1):
    response = get_completion(prompt)
    answers.append(response)

for x in answers:
    print(x)

    print("___________")

print('Correct output text:\n', correct_text)

___________
Output text:
```
The database stores painters, paintings, and galleries.
Painters have a number, name, and phone number. The painter's number is unique.
Paintings have a number, title, and price. The painting's number is unique.
Galleries have a number, owner, phone number, commission rate, and address. The gallery's number is unique.
A painting is painted by one artist. The painting is exhibited in one gallery.
A gallery can exhibit multiple paintings. Each painting is exhibited in only one gallery.
A painting is painted by one painter. A painter can paint multiple paintings.
```
___________
Correct output text:
 
The database must store painters, paintings, and galleries.
Painters have a number, name, and phone number. The painter’s number is unique.
Paintings have a number, title and price. The painting’s number is unique.
Galleries have a number, owner, phone number, commission rate and address. The gallerie’s number is unique.
A painting is painted by a unique artist, 

Few_shot prompting - attemp 2

Minor problem:
- doesn't remove repetitive sentence (not a problem since repetitions are removed in the arrays)

In [ ]:
text = """The database must store painters, paintings, and galleries information.
Painters have a unique number, Name, and phone number. Paintings have a unique number, title and price.
Galleries have a unique number, owner, phone number, commission rate and address.
A painting is painted by a particular artist, and that painting is exhibited in a particular gallery.
A gallery can exhibit many paintings, but each painting can be exhibited in only one gallery.
Similarity, a painting is painted by a single painter, but each painter can paint many paintings.
"""

prompt = f"""
The task is to extract an Entity-relationship diagram from natural language requirements.
To do that, these requirements should be simplified as much as possible for the extraction of entities,
attributes and relationships (i.e. short and simple sentences, explicit subjects, etc.).
Two examples of semplification are given below, where the 'input text' is the original text and the 'output text' is the simplified text:

Input text:
'
{ex1_input}
'

Output text:
'
{ex1_output}
'

Input text:
'
{ex2_input}
'

Output text:
'
{ex2_output}
'

Simplify as shown above the following text delimited by triple backticks, remembering to not remove any information, creating short and simple sentences and pay attention to not repeat any sentence if there are synonyms:

Text:
```{text}```
"""

answers = []
print("___________")
for i in range(1):
    response = get_completion(prompt)
    answers.append(response)

for x in answers:
    print(x)

    print("___________")

print('Correct output text:\n', correct_text)

___________
Output text:
```
The database stores painters, paintings, and galleries.
Painters have a number, name, and phone number. The painter's number is unique.
Paintings have a number, title, and price. The painting's number is unique.
Galleries have a number, owner, phone number, commission rate, and address. The gallery's number is unique.
A painting is created by one painter. The painting is displayed in one gallery.
A gallery can display multiple paintings. Each painting is displayed in only one gallery.
A painting is created by one painter. Each painter can create multiple paintings.
```
___________
Correct output text:
 
The database must store painters, paintings, and galleries.
Painters have a number, name, and phone number. The painter’s number is unique.
Paintings have a number, title and price. The painting’s number is unique.
Galleries have a number, owner, phone number, commission rate and address. The gallerie’s number is unique.
A painting is painted by a unique art

Few-shot prompting - attempt 2

Previous prompt with different input (testing purposes 2)

In [ ]:
correct_text = """
The database stores book, author, publisher and warehouse.
Every book has a title, isbn, year and price.
The book’s isbn is unique. Every author has an id, name, address and the URL.
Each author can write many books, and each book can have many authors.
Every publisher has an id, name, address, phone number and an URL. Books are stored at several warehouses.
A warehouse has a code, address and phone number. A book has only one publisher.
The warehouse stocks many books. A book may be stocked at multiple warehouses. A book has number of copies.
"""

In [ ]:
text = """
The database must store book, author, publisher and warehouse information.
For every book you must capture the title, isbn, year and price information.
The isbn value is unique for a book. For every author you must store an id, name, address and the URL of their homepage.
Each author can write many books, and each book can have many authors, for example.
For every publisher you must store an id, name, address, phone number and an URL of their website.
Books are stored at several warehouses, each of which has a code, address and phone number. A book has only one publisher.
The warehouse stocks many different books. A book may be stocked at multiple warehouses.
The database records the number of copies of a book stocked at various warehouses.
"""

prompt = f"""
The task is to extract an Entity-relationship diagram from natural language requirements.
To do that, these requirements should be simplified as much as possible for the extraction of entities,
attributes and relationships (i.e. short and simple sentences, explicit subjects, etc.).
Two examples of semplification are given below, where the 'input text' is the original text and the 'output text' is the simplified text:

Input text:
'
{ex1_input}
'

Output text:
'
{ex1_output}
'

Input text:
'
{ex2_input}
'

Output text:
'
{ex2_output}
'

Simplify as shown above the following text delimited by triple backticks, remembering to not remove any information, creating short and simple sentences and pay attention to not repeat any sentence if there are synonyms:

Text:
```{text}```
"""

answers = []
print("___________")
for i in range(1):
    response = get_completion(prompt)
    answers.append(response)

for x in answers:
    print(x)

    print("___________")

print('Correct output text:\n', correct_text)

___________
```
The database must store book, author, publisher and warehouse information.
A book has a title, isbn, year and price. The book's isbn is unique.
An author has an id, name, address and homepage URL. The author's id is unique.
An author can write many books. A book can have many authors.
A publisher has an id, name, address, phone number and website URL. The publisher's id is unique.
A book is published by a unique publisher.
A warehouse has a code, address and phone number. The warehouse's code is unique.
A warehouse can stock many books. A book can be stocked in many warehouses.
The database records the number of copies of a book in a warehouse.
```
___________
Correct output text:
 
The database stores book, author, publisher and warehouse.
Every book has a title, isbn, year and price.
The book’s isbn is unique. Every author has an id, name, address and the URL.
Each author can write many books, and each book can have many authors.
Every publisher has an id, name, addre

Few-shot prompting - attempt 2

Previous prompt with different input (testing purposes 2)

Problems:
- If there are words with '_' or '-', it would be better to modify them directly using GPT in the text to avoid extracting and modifying them later.

In [ ]:
correct_text = """
Physicians have ID and specialty. The physicians’ ID is unique. Patients have ID and name.
Physicians admits one or more patients. A patient is admitted by one physician.
A physician treats zero or more patients. A patient is treated by one or more physicians.
The patients’ ID is unique. A patient has one physician. If the patient is treated, it has one or more treatment details.
Treatment details have date, time and result.
"""

In [ ]:
text = """
A hospital has a large number of registered physicians. Attributes of PHYSICIANS include Physician_ID (the identifier) and Specialty.
Patients are admitted to the hospital by physicians. Attributes of PATIENT include Patient_ID (the identifier) and PatientName.
Any patient who is admitted must have exactly one admitting physician. A physician may optionally admit any number of patients.
Once admitted, a given patient must be treated by at least one physician.
A particular physician may treat any number of patients, or may not treat any patients.
Whenever a patient is treated by a physician, the hospital wishes to record the details of the treatment (Treatment_Detail).
Components of the Treatment_Detail include Date, Time, and Result.
"""

prompt = f"""
The task is to extract an Entity-relationship diagram from natural language requirements.
To do that, these requirements should be simplified as much as possible for the extraction of entities,
attributes and relationships (i.e. short and simple sentences, explicit subjects, etc.).
Two examples of semplification are given below, where the 'input text' is the original text and the 'output text' is the simplified text:

Input text:
'
{ex1_input}
'

Output text:
'
{ex1_output}
'

Input text:
'
{ex2_input}
'

Output text:
'
{ex2_output}
'

Simplify as shown above the following text delimited by triple backticks, remembering to not remove any information, creating short and simple sentences and pay attention to not repeat any sentence if there are synonyms:

Text:
```{text}```
"""

answers = []
print("___________")
for i in range(1):
    response = get_completion(prompt)
    answers.append(response)

for x in answers:
    print(x)

    print("___________")

print('Correct output text:\n', correct_text)


# A hospital has a large number of registered physicians. Attributes of PHYSICIANS include Physician_ID (the identifier) and Specialty.
# Patients are admitted to the hospital by physicians. Attributes of PATIENT include Patient_ID (the identifier) and PatientName.
# Any patient who is admitted must have exactly one admitting physician. A physician may optionally admit any number of patients.
# Once admitted, a given patient must be treated by at least one physician. A particular physician may treat any number of patients, or may not treat any patients.
# Whenever a patient is treated by a physician, the hospital wishes to record the details of the treatment (Treatment_Detail). Components of the Treatment_Detail include Date, Time, and Result.

___________
```
A hospital has many registered physicians. A physician has a Physician_ID and Specialty. The Physician_ID is unique.
Patients are admitted to the hospital by physicians. A patient has a Patient_ID and PatientName. The Patient_ID is unique.
A patient is admitted by exactly one physician. A physician can admit zero or many patients.
A patient is treated by at least one physician. A physician can treat zero or many patients.
The hospital records the treatment details. The treatment detail includes Date, Time, and Result.
```
___________
Correct output text:
 
Physicians have ID and specialty. The physicians’ ID is unique. Patients have ID and name.
Physicians admits one or more patients. A patient is admitted by one physician.
A physician treats zero or more patients. A patient is treated by one or more physicians.
The patients’ ID is unique. A patient has one physician. If the patient is treated, it has one or more treatment details.
Treatment details have date, time and 

Few-shot prompting - attempt 3

In [ ]:
text = """
A hospital has a large number of registered physicians. Attributes of PHYSICIANS include Physician_ID (the identifier) and Specialty.
Patients are admitted to the hospital by physicians. Attributes of PATIENT include Patient_ID (the identifier) and PatientName.
Any patient who is admitted must have exactly one admitting physician. A physician may optionally admit any number of patients.
Once admitted, a given patient must be treated by at least one physician.
A particular physician may treat any number of patients, or may not treat any patients.
Whenever a patient is treated by a physician, the hospital wishes to record the details of the treatment (Treatment_Detail).
Components of the Treatment_Detail include Date, Time, and Result.
"""

prompt = f"""
The task is to extract an Entity-relationship diagram from natural language requirements.
To do that, these requirements should be simplified as much as possible for the extraction of entities,
attributes and relationships (i.e. short and simple sentences, explicit subjects, etc.).
Two examples of semplification are given below, where the 'input text' is the original text and the 'output text' is the simplified text:

Input text:
'
{ex1_input}
'

Output text:
'
{ex1_output}
'

Input text:
'
{ex2_input}
'

Output text:
'
{ex2_output}
'

Simplify as shown above the following text delimited by triple backticks, remembering to not remove any information, creating short and simple sentences and pay attention to not repeat any sentence if there are synonyms.
If there are words composed with '-' or '_' or in camel case, maintain only the main token (i.e. the possessed token, e.g. in 'person_id', maintain id):

Text:
```{text}```
"""

answers = []
print("___________")
for i in range(1):
    response = get_completion(prompt)
    answers.append(response)

for x in answers:
    print(x)

    print("___________")

print('Correct output text:\n', correct_text)

___________
```
A hospital has many physicians. Physicians have an ID and Specialty. The physician's ID is unique.
Patients are admitted to the hospital by physicians. Patients have an ID and name. The patient's ID is unique.
A patient is admitted by exactly one physician. A physician can admit zero or many patients.
A patient is treated by at least one physician. A physician can treat zero or many patients.
The hospital records the treatment details. The treatment detail includes date, time, and result.
```
___________
Correct output text:
 
Physicians have ID and specialty. The physicians’ ID is unique. Patients have ID and name.
Physicians admits one or more patients. A patient is admitted by one physician.
A physician treats zero or more patients. A patient is treated by one or more physicians.
The patients’ ID is unique. A patient has one physician. If the patient is treated, it has one or more treatment details.
Treatment details have date, time and result.



Few-shot prompting - attemp 3

Previous prompt with different input (testing purposes)

In [ ]:
correct_text = """
Each member has an ID, name, zip, payment date. The member’s ID is unique. Each membership type has an ID, name and price.
The membership type’s ID is unique. Each pass category has an ID, name and price. The ID is unique. Each one day pass has the ID and date.
The one day pass’ ID is unique. Each merchandise item has an ID, name and price. The merchandise item’s ID is unique.
Each sale transaction has an ID and date. The transaction’s ID is unique. A member pays for one membership type.
A membership type has one or more members. Each member can buy zero or more one day passes. Each day pass is bought by one member.
Each one day pass belongs to one pass category. Each pass category can have zero or more one day passes. Each sale transaction involves one member.
Each member can be involved in zero or more sale transactions. Each merchandise item is sold through one or more sale transaction.
Each sale transaction involves one or more merchandise item. Every time a merchandise item is sold via a sale transaction, we keep track of the sold quantity.
"""

In [ ]:
text = """
For each MEMBER we keep track of the unique MemdID, a well as Name, Zip, and the Date the membership was paid.
For each MEMBERSHIP type we keep track of the unique Mid, as well as MName and Price.
For each PASS CATEGORY we keep track of the unique PassCatID, as well as PCName and Price.
For each ONE DAY PASS we keep track of the unique PassID and Date.
For each MERCHANDISE item we keep track of the unique MrchID, as well as Name and Price.
For each sale TRANSACTION we keep track of the unique Tid and Date.
Each member pays for exactly one membership type; each membership type has at least one member but can have many members.
Each member can buy many day passes but does not have to buy any, each day pass was bought by exactly one member.
Each day pass belongs to exactly one pass category; a pass category can have many individual day passes issued for it but does not have to have any.
Each sale transaction involves exactly one member; each member can be involved in many sale transactions but does not have to be involved in any.
Each merchandise item is sold via at least one sale transaction but it can be sold via many sale transactions; each sale transaction involves at least one merchandise item but can involve many merchandise items.
Every time a merchandise item is sold via a sale transaction, we keep track of the quantity.
"""

prompt = f"""
The task is to extract an Entity-relationship diagram from natural language requirements.
To do that, these requirements should be simplified as much as possible for the extraction of entities,
attributes and relationships (i.e. short and simple sentences, explicit subjects, etc.).
Two examples of semplification are given below, where the 'input text' is the original text and the 'output text' is the simplified text:

Input text:
'
{ex1_input}
'

Output text:
'
{ex1_output}
'

Input text:
'
{ex2_input}
'

Output text:
'
{ex2_output}
'

Simplify as shown above the following text delimited by triple backticks, remembering to not remove any information, creating short and simple sentences and pay attention to not repeat any sentence if there are synonyms.
If there are words composed with '-' or '_' or in camel case, maintain only the main token (i.e. the possessed token, e.g. in 'person_id', maintain id):

Text:
```{text}```
"""

answers = []
print("___________")
for i in range(1):
    response = get_completion(prompt)
    answers.append(response)

for x in answers:
    print(x)

    print("___________")

print('Correct output text:\n', correct_text)

___________
```
Each member has a MemdID, Name, Zip, and Date. The MemdID is unique.
Each membership type has a Mid, MName, and Price. The Mid is unique.
Each pass category has a PassCatID, PCName, and Price. The PassCatID is unique.
Each one day pass has a PassID and Date. The PassID is unique.
Each merchandise item has a MrchID, Name, and Price. The MrchID is unique.
Each sale transaction has a Tid and Date. The Tid is unique.
A member pays for exactly one membership type. A membership type can have one or many members.
A member can buy zero or many day passes. Each day pass is bought by a unique member.
A day pass belongs to a unique pass category. A pass category can have zero or many day passes.
A sale transaction involves a unique member. A member can be involved in zero or many sale transactions.
A merchandise item is sold via one or many sale transactions. A sale transaction involves one or many merchandise items.
Each sale transaction has a quantity.
```
___________
Correct ou

Few-shot prompting - attemp 4 - final one

derived from prompt 3, short modification in the task explanation

In [ ]:
text = """
For each MEMBER we keep track of the unique MemdID, a well as Name, Zip, and the Date the membership was paid.
For each MEMBERSHIP type we keep track of the unique Mid, as well as MName and Price.
For each PASS CATEGORY we keep track of the unique PassCatID, as well as PCName and Price.
For each ONE DAY PASS we keep track of the unique PassID and Date.
For each MERCHANDISE item we keep track of the unique MrchID, as well as Name and Price.
For each sale TRANSACTION we keep track of the unique Tid and Date.
Each member pays for exactly one membership type; each membership type has at least one member but can have many members.
Each member can buy many day passes but does not have to buy any, each day pass was bought by exactly one member.
Each day pass belongs to exactly one pass category; a pass category can have many individual day passes issued for it but does not have to have any.
Each sale transaction involves exactly one member; each member can be involved in many sale transactions but does not have to be involved in any.
Each merchandise item is sold via at least one sale transaction but it can be sold via many sale transactions; each sale transaction involves at least one merchandise item but can involve many merchandise items.
Every time a merchandise item is sold via a sale transaction, we keep track of the quantity.
"""

prompt = f"""
The task is to extract an Entity-relationship diagram from natural language requirements.
To do that, these requirements should be simplified as much as possible for the extraction of entities,
attributes and relationships (i.e. short and simple sentences, explicit subjects, etc.).
Two examples of semplification are given below, where the 'input text' is the original text and the 'output text' is the simplified text:

Input text:
'
{ex1_input}
'

Output text:
'
{ex1_output}
'

Input text:
'
{ex2_input}
'

Output text:
'
{ex2_output}
'

Simplify as shown above the following text delimited by triple backticks, remembering to not remove any information, creating short and simple sentences and pay attention to not repeat any sentence if there are synonyms.
If there are words composed with '-' or '_' or in camel case, maintain only the main token (i.e. the possessed token, e.g. in 'person_id', maintain id). Return as output only the modified text:

Text:
```{text}```
"""

answers = []
print("___________")
for i in range(1):
    response = get_completion(prompt)
    answers.append(response)

for x in answers:
    print(x)

    print("___________")

print('Correct output text:\n', correct_text)

___________
```
Each MEMBER has a MemdID, Name, Zip, and Date. The MemdID is unique.
Each MEMBERSHIP type has a Mid, MName and Price. The Mid is unique.
Each PASS CATEGORY has a PassCatID, PCName and Price. The PassCatID is unique.
Each ONE DAY PASS has a PassID and Date. The PassID is unique.
Each MERCHANDISE item has a MrchID, Name and Price. The MrchID is unique.
Each TRANSACTION has a Tid and Date. The Tid is unique.
A member pays for exactly one membership type. A membership type can have one or many members.
A member can buy many day passes. Each day pass is bought by exactly one member.
A day pass belongs to exactly one pass category. A pass category can have many day passes.
A transaction involves exactly one member. A member can be involved in many transactions.
A merchandise item is sold via one or many transactions. A transaction involves one or many merchandise items.
Each time a merchandise item is sold, we keep track of the quantity.
```
___________
Correct output text:
 